In [1]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cuda:0,floatX=float32"
#import theano
#print(theano.config.device, theano.config.floatX)

import keras
import keras_metrics
import sys
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
#from keras.applications import resnet
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



Using TensorFlow backend.


In [2]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:01:00.0, compute capability: 7.0
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device



In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16903439595751731400
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13642675103112376558
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 24301820314
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11808060847465704865
physical_device_desc: "device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:01:00.0, compute capability: 7.0"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12049550360335863450
physical_device_desc: "device: XLA_GPU device"
]


In [4]:
config = tf.ConfigProto( device_count = {'GPU': 2 } ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [5]:
K.set_learning_phase(1)

In [6]:
img_width, img_height = 71, 71
nb_train_samples = 26640
nb_validation_samples = 12630
epochs = 12
batch_size = 8
n_classes = 6

In [7]:
!ls /datasets/DTLD/DTLD_crop/new_test/colours/0 |wc -l

244


In [8]:
def build_model_Xception():
    input_shape=(img_width, img_height, 3)
    img_input = Input(shape=input_shape)  
    base_model = xception.Xception(include_top=False, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None)
    for layer in base_model.layers:
        layer.trainable = True
    x = base_model(img_input)
    x = Flatten()(x)
    pred_col = Dense(units=6, activation="sigmoid", name="pred_colors")(x)
    pred_num = Dense(units=5, activation="sigmoid", name="pred_numbers")(x)
    pred_pos = Dense(units=5, activation="sigmoid", name="pred_orientation")(x)
    for layer in base_model.layers:
        layer.trainable = True
    model = Model(inputs=img_input, outputs=[pred_pos, pred_col, pred_num], name='my_Xception')
                  #outputs=[pred_sig, pred_col, pred_pos], name='my_Xception')
    return model

In [9]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse'])

In [10]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepath="weights-improvement-triple-71-{epoch:02d}-aorient-{val_pred_orientation_acc:.2f}-acolor{val_pred_colors_acc:.2f}-anum{val_pred_numbers_acc:.2f}.hdf5"
#check = ModelCheckpoint(filepath, monitor = "val_pred_colors_loss", save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
check = ModelCheckpoint(filepath, save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, check]

In [11]:
with open("/home/z_andrei/datasets/new_annotations/all_images", 'rb') as f:
    x = pickle.load(f)
with open("/home/z_andrei/datasets/new_annotations/annotations", 'rb') as f:
    y = pickle.load(f)
with open("/home/z_andrei/datasets/new_annotations/all_images_test", 'rb') as f:
    x_test = pickle.load(f)
with open("//home/z_andrei/datasets/new_annotations/annotations_test", 'rb') as f:
    y_test = pickle.load(f)

In [12]:
'''
for i in range(10):
    img = x[i]
    cv2.imshow('image',img)
    cv2.waitKey(0)
cv2.destroyAllWindows()
'''

"\nfor i in range(10):\n    img = x[i]\n    cv2.imshow('image',img)\n    cv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [13]:
x = np.array(x)
x_test = np.array(x_test)
y=np.array([np.array(u) for u in y])
y1, y2, y3 = y[:, 0], y[:, 1], y[:, 2]
y1=np.array([np.array(u) for u in y1])
y2=np.array([np.array(u) for u in y2])
y3=np.array([np.array(u) for u in y3])
y_test=np.array([np.array(u) for u in y_test])
y1_test, y2_test, y3_test = y_test[:, 0], y_test[:, 1], y_test[:, 2]
y1_test=np.array([np.array(u) for u in y1_test])
y2_test=np.array([np.array(u) for u in y2_test])
y3_test=np.array([np.array(u) for u in y3_test])
y1_test[:10]

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [14]:
stringlist = []
model.summary(print_fn=lambda x: stringlist.append(x))
short_model_summary = "\n".join(stringlist)
print(short_model_summary)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 71, 71, 3)    0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 3, 3, 2048)   20861480    input_1[0][0]                    
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 18432)        0           xception[1][0]                   
__________________________________________________________________________________________________
pred_orientation (Dense)        (None, 5)            92165       flatten_1[0][0]                  
__________________________________________________________________________________________________
pred_color

In [15]:
print(x.shape, y1.shape, y2.shape, y3.shape)

(26832, 71, 71, 3) (26832, 5) (26832, 6) (26832, 5)


In [16]:
model_history = model.fit(
                            x,
                            [y1, y2, y3],
                            epochs=epochs, 
                            batch_size=batch_size,
                            validation_data=(x_test, [y1_test, y2_test, y3_test]),
                            callbacks=callbacks_list
                        )
#hist = model.fit(x, [y_gender,y_age], epochs=epochs, batch_size=batch_size, validation_data=(dataVal, [labelsGenderVal,labelsAgeVal]), callbacks=callbacks)
    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 26832 samples, validate on 6708 samples
Epoch 1/12
26832/26832 [==============================] - 258s 10ms/step - loss: 1.9998 - pred_orientation_loss: 0.5744 - pred_colors_loss: 0.7775 - pred_numbers_loss: 0.6479 - pred_orientation_acc: 0.8792 - pred_orientation_mean_squared_error: 0.0409 - pred_colors_acc: 0.7570 - pred_colors_mean_squared_error: 0.0792 - pred_numbers_acc: 0.8473 - pred_numbers_mean_squared_error: 0.0512 - val_loss: 1.9088 - val_pred_orientation_loss: 0.4774 - val_pred_colors_loss: 0.8503 - val_pred_numbers_loss: 0.5811 - val_pred_orientation_acc: 0.9113 - val_pred_orientation_mean_squared_error: 0.0334 - val_pred_colors_acc: 0.7199 - val_pred_colors_mean_squared_error: 0.1134 - val_pred_numbers_acc: 0.8798 - val_pred_numbers_mean_squared_error: 0.0410
Epoch 2/12
26832/26832 [==============================] - 247s 9ms/step - loss: 1.6693 - pred_orientation_loss: 

In [17]:
callbacks_list

In [18]:
model = load_model('weights-improvement-71-08-0.96.hdf5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse', precision, recall])

NameError: name 'precision' is not defined

In [ ]:
evaluation = model.evaluate_generator(validation_generator, verbose=1)

In [ ]:
predictions = model.predict_generator(validation_generator, verbose=1)

In [ ]:
np.argmax(predictions[0:224], axis=1)

In [ ]:
model.metrics_names

In [ ]:
evaluation

In [ ]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [ ]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall